# Question 7

In [ ]:
import numpy as np
from tqdm import tqdm # Nice Progress bar for loops
import matplotlib.pyplot as plt
from Rankine_solver import define_empty_variables, solve_r_rankine_cycle
from Rankine_solver import display_tables, plot_Ts_cycle
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [ ]:
def get_m_dot_from_T(G,TC,TH,T2, T4,qh,qc):
    if T2 >= TC :
        return 0  # No heat absorption or rejection possible if working fluid(T2) is hotter than cold reservoir (TC)
    m_dot_H = 0
    if abs(qh) > 1e-5 :
        m_dot_H = (G * 1e-3 * np.abs(TH - T4)) / qh

    m_dot_C = 0
    if abs(qc) > 1e-5 :
        m_dot_C = (G * 1e-3 * np.abs(TC - T2)) / qc

    m_dot = max(min(m_dot_C, m_dot_H), 0)
    return m_dot



def get_t2_t4(T_h, T_c_vals, Q_h_vals):
    G_heatp = 4895.423478997306
    wc_vals = np.zeros_like(T_c_vals)

    for i in tqdm(range(len(T_c_vals)), desc="Computing Wc"):
        T_c = T_c_vals[i]
        Q_h = Q_h_vals[i]

        T2_range = np.linspace(T_c - 15, T_c - 0.1, 20)
        T4_range = np.linspace(T_h + 0.1, T_h + 15, 20)
        best_Wc = np.inf
        best_m_dot = 0

        for T2 in T2_range:
            for T4 in T4_range:
                vars = define_empty_variables()
                vars['2']['T'] = T2
                vars['4']['T'] = T4
                vars['Qh_dot'] = Q_h

                solved = solve_r_rankine_cycle(vars)
                m_dot = solved['m_dot']
                Wc_dot = solved['Wc_dot']
                qh = solved['qh']
                qc = solved['qc']
                m_dot_max = get_m_dot_from_T(G_heatp, T_c, T_h, T2, T4, qh, qc)

                if m_dot <= m_dot_max and Wc_dot < best_Wc:
                    best_Wc = Wc_dot
                    best_m_dot = m_dot

        wc_vals[i] = best_Wc

    return wc_vals



In [ ]:
#part 2
G = 682.27  # W/K
T_h = 22
T_c = np.linspace(5,22,15)
Q_h_vals = np.zeros_like(T_c) 

for i in range(len(T_c)):
    Q_h_vals[i] = G * (T_h - T_c[i]) / 1000



wc = get_t2_t4(T_h,T_c,Q_h_vals)

plt.figure(figsize=(8, 5))
plt.plot(T_c, wc, marker='o')
plt.title("Optimized compressor work vs exterior temperature")
plt.xlabel("Exterior Temperature (ºC)")
plt.ylabel("Compressor work (kW)")
plt.grid(True)
plt.show()

